In [1]:
from pathlib import Path

import numba as nb
import numpy as np
import scipy.stats as stats

from natsort import natsorted

import matplotlib.pyplot as plt


from threeML import *

plt.style.use("mike")
import warnings
warnings.simplefilter("ignore")


%matplotlib widget



In [2]:
spi = OGIPLike("spi", observation="fit_GRB120711A_BB_SwiftPosition_spec/spectra_GRB.fits",
               response="fit_GRB120711A_BB_SwiftPosition_spec/spectral_response.rmf.fits")



[WARNING ] Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.
[WARNING ] FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.
[WARNING ] The response is in an extension called MATRIX, which usually means you also need an ancillary file (ARF) which you didn't provide. You should refer to the documentation  of the instrument and make sure you don't need an ARF.
[INFO    ] Auto-probed noise models:
[INFO    ] - observation: gaussian
[INFO    ] - background: None


In [3]:
spi.view_count_spectrum();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
band = Band()
band.alpha.prior = Truncated_gaussian(lower_bound=-1.5, upper_bound=1, mu=-1, sigma=0.5)
band.beta.prior = Truncated_gaussian(lower_bound=-5, upper_bound=-1.6, mu=-2, sigma=0.5)
band.xp.prior = Log_normal(mu=2, sigma=1)
band.xp.bounds = (0, None)
band.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
ps = PointSource("grb", 0, 0, spectral_shape=band)
band_model = Model(ps)

In [5]:
bayes = BayesianAnalysis(band_model, DataList(spi))

In [8]:
bayes.set_sampler("multinest")
bayes.sampler.setup(n_live_points=400)
_ = bayes.sample()

[INFO    ] sampler set to multinest
  analysing data from chains/fit-.txt
Maximum a posteriori probability (MAP) point:



,result,unit
parameter,,
grb.spectrum.main.Band.K,(3.79 +/- 0.04) x 10^-2,1 / (cm2 keV s)
grb.spectrum.main.Band.alpha,(-8.22 -0.06 +0.05) x 10^-1,
grb.spectrum.main.Band.xp,(6.2 +/- 0.4) x 10^2,keV
grb.spectrum.main.Band.beta,-2.137 +/- 0.004,



Values of -log(posterior) at the minimum:



,-log(posterior)
spi,-31.208039
total,-31.208039



Values of statistical measures:



,statistical measures
AIC,71.897560
BIC,76.279022
DIC,68.232395
PDIC,0.887534
log(Z),-24.749211


In [12]:
bayes.restore_median_fit()

display_spectrum_model_counts(bayes);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plot_spectra(bayes.results, flux_unit="erg2/(cm2 s keV)");

processing MLE analyses: 0it [00:00, ?it/s]

processing Bayesian analyses:   0%|          | 0/1 [00:00<?, ?it/s]

Propagating errors:   0%|          | 0/100 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
bayes.results.corner_plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …